## Combbine all diagnosis

This method take diagnosis from images, clinical, and diagnosis sheet, and creates one ground truth (where all three agree) and one majority vote (where two agree) diagnosis files. 

In [506]:
import pandas as pd
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
img = pd.read_csv("all_img_try1_10_31_2021.csv")
new = pd.DataFrame.from_dict(d, orient='index').reset_index()
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]

In [507]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [508]:
clinical["year"] = clinical["EXAMDATE"].str[:4]

In [509]:
clinical["Subject"] = clinical["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")

In [510]:
c = comb.merge(clinical, on = ["RID", "Phase"])

In [511]:
c = c.drop("Subject", axis =1)

In [512]:
c = c.rename(columns = {"PTID":"Subject"})

In [513]:
c

,RID,Subject,Phase,SUBJECT_KEY,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp,year
0,2,011_S_0002,ADNI1,ADNI_011_S_0002,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0,2008
1,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,16.0,2,5,74.4394,1.0,0.277,0.254,0.293,NaN,2021-04-14 13:05:56.0,2005
2,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,1,16.0,2,5,74.9295,1.0,0.237,0.278,0.535,-0.333,2021-04-14 13:05:56.0,2006
3,2,011_S_0002,ADNI1,ADNI_011_S_0002,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0,2008
4,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,16.0,2,5,74.4394,1.0,0.277,0.254,0.293,NaN,2021-04-14 13:05:56.0,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49712,6851,168_S_6851,ADNI3,NaN,sc,sc,2020-02-05,2,18.0,2,5,76.5996,2.0,-0.180,NaN,NaN,NaN,2021-04-14 13:06:13.0,2020
49713,4580,013_S_4580,ADNI3,ADNI_013_s_4580,init,m72,2018-01-24,2,16.0,2,5,75.6496,1.0,0.821,0.441,0.956,0.483,2021-04-14 13:06:06.0,2018
49714,6841,035_S_6841,ADNI3,NaN,sc,sc,2019-12-04,2,14.0,2,5,74.7598,1.0,0.870,NaN,NaN,NaN,2021-04-14 13:06:13.0,2019
49715,6855,032_S_6855,ADNI3,NaN,sc,sc,2020-02-05,1,16.0,2,5,74.6804,3.0,-0.814,NaN,NaN,NaN,2021-04-14 13:06:13.0,2020


In [514]:
img["year"] = img["Acq Date"].str[5:].str.replace("/", "")

In [515]:
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])

In [516]:
c["DX"] = c["DX"] -1

In [517]:
new[0] = new[0].astype(int) -1

In [518]:
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})

In [519]:
m = new.merge(c, on = "Subject", how = "outer").merge(img, on = "Subject", how = "outer")

In [520]:
img[img["Subject"] == "003_S_0908"][[ "Group"]]

,Group
3648,1
3649,1
3650,1
3651,1
3652,1
3653,1
3654,1
3655,1


In [521]:
m[["GroupN", "DX", "Group"]]

,GroupN,DX,Group
0,1.0,0.0,NaN
1,1.0,0.0,NaN
2,1.0,0.0,NaN
3,1.0,0.0,NaN
4,1.0,0.0,NaN
...,...,...,...
162491,1.0,NaN,NaN
162492,1.0,NaN,NaN
162493,1.0,NaN,NaN
162494,NaN,NaN,NaN


In [523]:
m = m[["Subject", "GroupN", "Group", "DX", "Phase"]].drop_duplicates()

In [524]:
m = m.dropna(subset = ["GroupN", "Group", "DX"], how="all").drop_duplicates()
m

,Subject,GroupN,Group,DX,Phase
0,011_S_0002,1.0,NaN,0.0,ADNI1
9,011_S_0002,1.0,NaN,0.0,ADNIGO
10,011_S_0002,1.0,NaN,0.0,ADNI2
13,011_S_0002,1.0,NaN,1.0,ADNI2
15,011_S_0002,1.0,NaN,NaN,ADNI2
...,...,...,...,...,...
162489,037_S_7012,0.0,NaN,NaN,NaN
162490,035_S_7019,1.0,NaN,NaN,NaN
162491,035_S_7021,1.0,NaN,NaN,NaN
162492,141_S_7013,1.0,NaN,NaN,NaN


In [525]:
m.loc[m["DX"].isna() & m["Group"].isna(), "Group"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]
m.loc[m["DX"].isna() & m["Group"].isna(), "DX"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]

In [526]:
m

,Subject,GroupN,Group,DX,Phase
0,011_S_0002,1.0,NaN,0.0,ADNI1
9,011_S_0002,1.0,NaN,0.0,ADNIGO
10,011_S_0002,1.0,NaN,0.0,ADNI2
13,011_S_0002,1.0,NaN,1.0,ADNI2
15,011_S_0002,1.0,1.0,NaN,ADNI2
...,...,...,...,...,...
162489,037_S_7012,0.0,0.0,NaN,NaN
162490,035_S_7019,1.0,1.0,NaN,NaN
162491,035_S_7021,1.0,1.0,NaN,NaN
162492,141_S_7013,1.0,1.0,NaN,NaN


In [527]:
m1 = m[m["GroupN"] == m["Group"]]
m3 = m[m["GroupN"] == m["DX"]]
m4 = m[m["Group"] == m["DX"]]
m2 = m1[m1["Group"] == m1["DX"]]

In [528]:
m1 = m1[["Subject", "GroupN", "Group", "DX", "Phase"]]
m1

,Subject,GroupN,Group,DX,Phase
15,011_S_0002,1.0,1.0,NaN,ADNI2
202,023_S_0031,0.0,0.0,0.0,ADNI1
526,023_S_0031,0.0,0.0,0.0,ADNIGO
535,023_S_0031,0.0,0.0,0.0,ADNI2
679,023_S_0031,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
162489,037_S_7012,0.0,0.0,NaN,NaN
162490,035_S_7019,1.0,1.0,NaN,NaN
162491,035_S_7021,1.0,1.0,NaN,NaN
162492,141_S_7013,1.0,1.0,NaN,NaN


In [529]:
m1.loc[m1["DX"].isna(), "DX"] = m1.loc[m1["DX"].isna(), "Group"]

In [530]:
m1

,Subject,GroupN,Group,DX,Phase
15,011_S_0002,1.0,1.0,1.0,ADNI2
202,023_S_0031,0.0,0.0,0.0,ADNI1
526,023_S_0031,0.0,0.0,0.0,ADNIGO
535,023_S_0031,0.0,0.0,0.0,ADNI2
679,023_S_0031,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
162489,037_S_7012,0.0,0.0,0.0,NaN
162490,035_S_7019,1.0,1.0,1.0,NaN
162491,035_S_7021,1.0,1.0,1.0,NaN
162492,141_S_7013,1.0,1.0,1.0,NaN


In [531]:
m3 = m3[["Subject", "GroupN", "Group", "DX", "Phase"]]
m3

,Subject,GroupN,Group,DX,Phase
13,011_S_0002,1.0,NaN,1.0,ADNI2
40,011_S_0002,1.0,NaN,1.0,ADNI3
41,011_S_0003,2.0,NaN,2.0,ADNI1
83,011_S_0008,1.0,NaN,1.0,ADNI2
99,011_S_0005,0.0,NaN,0.0,ADNI1
...,...,...,...,...,...
162348,013_S_6725,1.0,NaN,1.0,ADNI3
162351,168_S_6851,1.0,NaN,1.0,ADNI3
162363,035_S_6841,0.0,NaN,0.0,ADNI3
162365,032_S_6855,2.0,NaN,2.0,ADNI3


In [532]:
m3.loc[m3["Group"].isna(), "Group"] = m3.loc[m3["Group"].isna(), "GroupN"]

In [533]:
m4 = m4[["Subject", "GroupN", "Group", "DX", "Phase"]]
m4

,Subject,GroupN,Group,DX,Phase
202,023_S_0031,0.0,0.0,0.0,ADNI1
526,023_S_0031,0.0,0.0,0.0,ADNIGO
535,023_S_0031,0.0,0.0,0.0,ADNI2
679,023_S_0031,0.0,0.0,0.0,ADNI3
731,011_S_0021,0.0,0.0,0.0,ADNI1
...,...,...,...,...,...
159256,009_S_5252,2.0,2.0,2.0,ADNI2
159280,016_S_5251,2.0,2.0,2.0,ADNI2
159572,135_S_5275,2.0,2.0,2.0,ADNI2
159839,082_S_5184,2.0,2.0,2.0,ADNI2


In [534]:
m4[m4["GroupN"] != m4["DX"]]

,Subject,GroupN,Group,DX,Phase
1247,023_S_0042,2.0,1.0,1.0,ADNI1
1903,007_S_0041,2.0,1.0,1.0,ADNI1
2682,023_S_0061,2.0,0.0,0.0,ADNI1
3114,023_S_0061,2.0,0.0,0.0,ADNIGO
3126,023_S_0061,2.0,0.0,0.0,ADNI2
...,...,...,...,...,...
132137,941_S_4100,1.0,0.0,0.0,ADNI2
135522,037_S_4071,1.0,0.0,0.0,ADNI2
142357,013_S_4579,1.0,0.0,0.0,ADNI2
144411,016_S_4638,1.0,0.0,0.0,ADNI2


In [535]:
m2[["Subject", "GroupN", "Group", "DX", "Phase"]]

,Subject,GroupN,Group,DX,Phase
202,023_S_0031,0.0,0.0,0.0,ADNI1
526,023_S_0031,0.0,0.0,0.0,ADNIGO
535,023_S_0031,0.0,0.0,0.0,ADNI2
679,023_S_0031,0.0,0.0,0.0,ADNI3
731,011_S_0021,0.0,0.0,0.0,ADNI1
...,...,...,...,...,...
159256,009_S_5252,2.0,2.0,2.0,ADNI2
159280,016_S_5251,2.0,2.0,2.0,ADNI2
159572,135_S_5275,2.0,2.0,2.0,ADNI2
159839,082_S_5184,2.0,2.0,2.0,ADNI2


In [551]:
m5 = pd.concat([m1,m3,m4])
i = m5[m5["Group"] == m5["GroupN"]]
i = i[i["Group"] == i["DX"]]

In [552]:
i = i.drop_duplicates()

In [553]:
i

,Subject,GroupN,Group,DX,Phase
15,011_S_0002,1.0,1.0,1.0,ADNI2
202,023_S_0031,0.0,0.0,0.0,ADNI1
526,023_S_0031,0.0,0.0,0.0,ADNIGO
535,023_S_0031,0.0,0.0,0.0,ADNI2
679,023_S_0031,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
162348,013_S_6725,1.0,1.0,1.0,ADNI3
162351,168_S_6851,1.0,1.0,1.0,ADNI3
162363,035_S_6841,0.0,0.0,0.0,ADNI3
162365,032_S_6855,2.0,2.0,2.0,ADNI3


In [554]:
i[["Subject", "Group", "Phase"]].to_csv("ground_truth.csv")

In [540]:
m.update(m5[~m5.index.duplicated(keep='first')])

In [541]:
m

,Subject,GroupN,Group,DX,Phase
0,011_S_0002,1.0,NaN,0.0,ADNI1
9,011_S_0002,1.0,NaN,0.0,ADNIGO
10,011_S_0002,1.0,NaN,0.0,ADNI2
13,011_S_0002,1.0,1.0,1.0,ADNI2
15,011_S_0002,1.0,1.0,1.0,ADNI2
...,...,...,...,...,...
162489,037_S_7012,0.0,0.0,0.0,NaN
162490,035_S_7019,1.0,1.0,1.0,NaN
162491,035_S_7021,1.0,1.0,1.0,NaN
162492,141_S_7013,1.0,1.0,1.0,NaN


In [542]:
indexes = m.index

In [543]:
m["GROUP"] = -1

In [544]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["Group"]):
        val = row["GroupN"]
        
        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["DX"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val
        
    elif (row["Group"] == row["DX"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val
        

In [545]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

,Subject,GroupN,Group,DX,Phase
15,011_S_0002,1.0,1.0,1.0,ADNI2
202,023_S_0031,0.0,0.0,0.0,ADNI1
526,023_S_0031,0.0,0.0,0.0,ADNIGO
535,023_S_0031,0.0,0.0,0.0,ADNI2
679,023_S_0031,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
132137,941_S_4100,1.0,0.0,0.0,ADNI2
135522,037_S_4071,1.0,0.0,0.0,ADNI2
142357,013_S_4579,1.0,0.0,0.0,ADNI2
144411,016_S_4638,1.0,0.0,0.0,ADNI2


In [546]:
m

,Subject,GroupN,Group,DX,Phase,GROUP
0,011_S_0002,1.0,NaN,0.0,ADNI1,-1
9,011_S_0002,1.0,NaN,0.0,ADNIGO,-1
10,011_S_0002,1.0,NaN,0.0,ADNI2,-1
13,011_S_0002,1.0,1.0,1.0,ADNI2,1
15,011_S_0002,1.0,1.0,1.0,ADNI2,1
...,...,...,...,...,...,...
162489,037_S_7012,0.0,0.0,0.0,NaN,0
162490,035_S_7019,1.0,1.0,1.0,NaN,1
162491,035_S_7021,1.0,1.0,1.0,NaN,1
162492,141_S_7013,1.0,1.0,1.0,NaN,1


In [547]:
m[m["GROUP"] != -1]

,Subject,GroupN,Group,DX,Phase,GROUP
13,011_S_0002,1.0,1.0,1.0,ADNI2,1
15,011_S_0002,1.0,1.0,1.0,ADNI2,1
40,011_S_0002,1.0,1.0,1.0,ADNI3,1
41,011_S_0003,2.0,2.0,2.0,ADNI1,2
83,011_S_0008,1.0,1.0,1.0,ADNI2,1
...,...,...,...,...,...,...
162489,037_S_7012,0.0,0.0,0.0,NaN,0
162490,035_S_7019,1.0,1.0,1.0,NaN,1
162491,035_S_7021,1.0,1.0,1.0,NaN,1
162492,141_S_7013,1.0,1.0,1.0,NaN,1


In [548]:
m[["Subject", "GroupN", "Group", "DX", "GROUP", "Phase"]].to_csv("diagnosis_full.csv")